Topic: Challenge Set 1 Subject: Explore MTA turnstile data Date: 04/13/2018 Name: student name Worked with: other students' name Location: sea18_ds10/student_submissions/challenges/01-mta/shaikh_reshama/challenge_set_1_reshama.ipynb

In [104]:
import sys

In [105]:
# imports a library 'pandas', names it as 'pd'
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from IPython.display import Image

import pprint

# enables inline plots, without it plots don't show up in the notebook
%matplotlib inline

import dateutil.parser
from datetime import *

In [106]:
# various options in pandas
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 500)
pd.set_option('display.precision', 3)

In [130]:
#read data from this folder
df = pd.read_csv('turnstile_170902.csv')
mta = df.copy()

In [131]:
#Work with turnstile name as combo

mta['TURNSTILE'] = mta['C/A'] + mta['UNIT'] + mta['SCP']

mta = mta.drop('C/A', axis=1)
mta = mta.drop('UNIT', axis=1)
mta = mta.drop('SCP', axis=1)

In [132]:
#re-order cols

cols = mta.columns.tolist()
cols.insert(0, cols.pop(cols.index('TURNSTILE')))
mta = mta.reindex(columns= cols)

In [133]:
#drop useless data

mta = mta.drop('LINENAME', axis=1)
mta = mta.drop('DIVISION', axis=1)
mta = mta.drop('DESC', axis=1)

In [134]:
#remove exit as well

mta.columns = mta.columns.str.strip()
mta = mta.drop('EXITS', axis=1)

In [135]:
# make new columns, blank

leng = len(mta.index)

mta['ENT_COUNT'] = [0] * leng
mta['DATE_TIME'] = [0] * leng

In [136]:
#reorder the date time column

cols = mta.columns.tolist()
cols.insert(4, cols.pop(cols.index('DATE_TIME')))
mta = mta.reindex(columns= cols)

In [137]:
# Fill empty date time column with formula
# I dont think we will use this...

for i in range(leng):
    datetime = df.DATE[i]+ ' ' + df.TIME[i]      
    value = dateutil.parser.parse(datetime)
     
    mta.at[i,'DATE_TIME'] = value

In [159]:
#fill empty ent_count with formula

for i in range(leng):
    if i == 0: 
        mta.at[i,'ENT_COUNT'] = 0
    else:
        if mta.TURNSTILE[i] == mta.TURNSTILE[i-1]:
            diff = mta.ENTRIES[i] - mta.ENTRIES[i-1]
            # ignore any negative entry count or very large entry counts, turnstile malfunction
            # we should discuss a better 
            if diff < 0: diff = 0
            if diff > 10000: diff = 0
            mta.at[i,'ENT_COUNT'] = diff
        else:
            mta.at[i,'ENT_COUNT'] = 0

In [148]:
# mta = mta.drop('DATE', axis=1)
# mta = mta.drop('TIME', axis=1)
# mta = mta.drop('ENTRIES', axis=1)

In [149]:
mta.head()

,TURNSTILE,STATION,DATE,TIME,DATE_TIME,ENTRIES,ENT_COUNT
0,A002R05102-00-00,59 ST,08/26/2017,00:00:00,2017-08-26 00:00:00,6303202,0
1,A002R05102-00-00,59 ST,08/26/2017,04:00:00,2017-08-26 04:00:00,6303225,23
2,A002R05102-00-00,59 ST,08/26/2017,08:00:00,2017-08-26 08:00:00,6303237,12
3,A002R05102-00-00,59 ST,08/26/2017,12:00:00,2017-08-26 12:00:00,6303309,72
4,A002R05102-00-00,59 ST,08/26/2017,16:00:00,2017-08-26 16:00:00,6303508,199


In [162]:
# Check all negative and large numbers have been removed

mta.describe()

,ENTRIES,ENT_COUNT
count,1.998e+05,199761.000
mean,3.968e+07,160.429
std,2.078e+08,233.908
min,0.000e+00,0.000
25%,4.905e+05,7.000
50%,2.665e+06,67.000
75%,6.883e+06,222.000
max,2.130e+09,3239.000


In [ ]:
# Find some time of day trend?

In [ ]:
# Next step! Combine days together and analyze what locations are the most popular and one what days


#### Challenge 1   
  
- Open up a new IPython notebook
- Download a few MTA turnstile data files
- Open up a file, use csv reader to read it, make a python dict where
  there is a key for each (C/A, UNIT, SCP, STATION). These are the
  first four columns. The value for this key should be a list of
  lists. Each list in the list is the rest of the columns in a
  row. For example, one key-value pair should look like


{    ('A002','R051','02-00-00','LEXINGTON AVE'):
[
['NQR456', 'BMT', '01/03/2015', '03:00:00', 'REGULAR', '0004945474', '0001675324'],
['NQR456', 'BMT', '01/03/2015', '07:00:00', 'REGULAR', '0004945478', '0001675333'],
['NQR456', 'BMT', '01/03/2015', '11:00:00', 'REGULAR', '0004945515', '0001675364'],
...
]
}

#### Challenge 2

- Let's turn this into a time series.

 For each key (basically the control area, unit, device address and
 station of a specific turnstile), have a list again, but let the list
 be comprised of just the point in time and the count of entries.

This basically means keeping only the date, time, and entries fields
in each list. You can convert the date and time into datetime objects
-- That is a python class that represents a point in time. You can
combine the date and time fields into a string and use the
[dateutil](https://labix.org/python-dateutil) module to convert it
into a datetime object. For an example check
[this StackOverflow question](http://stackoverflow.com/questions/23385003/attributeerror-when-using-import-dateutil-and-dateutil-parser-parse-but-no).

Your new dict should look something like

{    ('A002','R051','02-00-00','LEXINGTON AVE'):
[
[datetime.datetime(2013, 3, 2, 3, 0), 3788],
[datetime.datetime(2013, 3, 2, 7, 0), 2585],
[datetime.datetime(2013, 3, 2, 12, 0), 10653],
[datetime.datetime(2013, 3, 2, 17, 0), 11016],
[datetime.datetime(2013, 3, 2, 23, 0), 10666],
[datetime.datetime(2013, 3, 3, 3, 0), 10814],
[datetime.datetime(2013, 3, 3, 7, 0), 10229],
...
],
....
}



#### Challenge 3

- These counts are for every n hours. (What is n?) We want total daily
  entries.

Now make it that we again have the same keys, but now we have a single
value for a single day, which is the total number of passengers that
entered through this turnstile on this day.


#### Challenge 4

We will plot the daily time series for a turnstile.

In ipython notebook, add this to the beginning of your next cell:

    %matplotlib inline

This will make your matplotlib graphs integrate nicely with the
notebook. To plot the time series, import matplotlib with

    import matplotlib.pyplot as plt

Take the list of [(date1, count1), (date2, count2), ...], for the
turnstile and turn it into two lists:
dates and counts. This should plot it:

    plt.figure(figsize=(10,3))
    plt.plot(dates,counts)
    

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

#### Challenge 5

- So far we've been operating on a single turnstile level, let's
  combine turnstiles in the same ControlArea/Unit/Station combo. There
  are some ControlArea/Unit/Station groups that have a single
  turnstile, but most have multiple turnstilea-- same value for the
  C/A, UNIT and STATION columns, different values for the SCP column.

We want to combine the numbers together -- for each
ControlArea/UNIT/STATION combo, for each day, add the counts from each
turnstile belonging to that combo.

#### Challenge 6

Similarly, combine everything in each station, and come up with a time
series of `[(date1, count1),(date2,count2),...]` type of time series
for each STATION, by adding up all the turnstiles in a station.

#### Challenge 7

Plot the time series for a station.

#### Challenge 8

- Make one list of counts for **one** week for one station. Monday's
count, Tuesday's count, etc. so it's a list of 7 counts.
Make the same list for another week, and another week, and another
week.
`plt.plot(week_count_list)` for every `week_count_list` you created
this way. You should get a rainbow plot of weekly commute numbers on
top of each other.

#### Challenge 9

- Over multiple weeks, sum total ridership for each station and sort
  them, so you can find out the stations with the highest traffic
  during the time you investigate

#### Challenge 10

- Make a single list of these total ridership values and plot it with

    plt.hist(total_ridership_counts)

to get an idea about the distribution of total ridership among
different stations.

This should show you that most stations have a small traffic, and the
histogram bins for large traffic volumes have small bars.

*Additional Hint*:

If you want to see which stations take the meat of the traffic, you
can sort the total ridership counts and make a `plt.bar` graph. For
this, you want to have two lists: the indices of each bar, and the
values. The indices can just be `0,1,2,3,...`, so you can do

    indices = range(len(total_ridership_values))
    plt.bar(indices, total_ridership_values)